In [ ]:
import os

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import numpy as np

from matplotlib.transforms import Bbox
from scipy.optimize import fmin_tnc, newton
from tqdm import tqdm

In [ ]:
# Figure text parameters
plt.rc('font', size=20)
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath,amssymb,bm,bbm,lmodern}')

In [ ]:
# Figure parameters
name_mapping = {
    'method_1': 'Sample Mean',
    'method_2': 'Affine Mean',
    'method_3': 'Affine Weighted',
    'method_4': 'Maximum Likelihood'
}
colours = {
    'method_1': "black",
    'method_2': "red",
    'method_3': "gold",
    'method_4': "aqua",
  }
markers = {
    'method_1': 'x',
    'method_2': '^',
    'method_3': 'v',
    'method_4': 'o'
  }
styles = {
    'method_1': '-',
    'method_2': ":",
    'method_3': '--',
    'method_4': '-.'
  }
linewidths = {
    "method_1": 2, 
    "method_2": 4.5, 
    "method_3": 3.5, 
    "method_4": 3.5
}

num_points_plotted = 50  # Too many points leads to jitter
alpha = 0.4  # Opacity

In [ ]:
N = 1000  # Num runs
n = 1000000  # Num ratings per run
p = 0.4  # True relevance

# True lambda distribution parameters. Replace if running weak bandwagon scenario.
a = 0.1
b = 0.95
graph_name = "strong_bw"

In [ ]:
# Plotting helper variables
x_values = np.arange(1, n + 1)
plot_idx = (np.unique(np.geomspace(1,n,num_points_plotted).astype(int))-1).tolist()

In [ ]:
# True lambda distribution: lambda_i = a + (1-a)*b**(i-1)
lambdas = a + (1 - a)*b**np.arange(n)

In [ ]:
# Generate the data by iterating over Equation 4
R = np.zeros((N,n))  # Keep track of individual ratings
M = np.zeros((N,n))  # Keep track of sample mean
ri = np.random.rand(N) < p  # First rating sampled from Bernoulli(p)
R[:,0] = ri
M[:,0] = ri
m = ri

# Iterate over timesteps (all runs in parallel)
for i in tqdm(range(1,n)):
    pri = lambdas[i]*p + (1 - lambdas[i])*m  # Calculate P(r_n=1|\bar{p}_{n-1})
    ri = np.random.rand(N) < pri  # Sample new rating
    m = (m*i + ri)/(i + 1)  # Update sample mean
    R[:,i] = ri  # Record the new rating
    M[:,i] = m  # Record the new sample mean

In [ ]:
# OPTIONAL: Only run this cell if using misestimated bandwagon values for estimators
a_hat, b_hat = 0.3305, 0.9185
lambdas_hat = a_hat + (1 - a_hat)*b_hat**np.arange(n)
lambdas = lambdas_hat

In [ ]:
# Sample mean as it was prior to the rating at the same index
M_prev = np.concatenate((np.ones((N,1)), M[:,:-1]), axis=1)

In [ ]:
# Affine estimators for each step
r_hat = (R - (1 - lambdas)*M_prev)/lambdas  # Equation 13

In [ ]:
# Affine mean estimator
p_hat = np.cumsum(r_hat, axis=1)/np.arange(1, n+1) 

In [ ]:
# Affine weighted estimator
p_hat_2 = np.cumsum(lambdas*r_hat, axis=1)/np.cumsum(lambdas)

In [ ]:
# OPTIONAL: Run this cell if training and saving the outputs for MLE (next cell) in a separate run.
del r_hat
del p_hat, p_hat_2

In [ ]:
# Maximum Likelihood - can get memory intensive
def f(p_current, r, m_shifted, lambdas):
    """Calculate the derivative of the log likelihood wrt. true relevance p estimate for a bandwagon process using
    m <= n first ratings.
    
    Args:
        p_current (np.array): Initial/current guess for the true relevance w/ shape (N, ).
        r (np.array): Observed ratings at each timestep w/ shape (N, m).
        m_shifted (np.array): Observed sample means before a given timestep w/ shape (N, m).
        lambdas (np.array): Estimated (or true) lambda values w/ shape (m, )
    """
    p_current = np.clip(p_current, 1e-8, 1-1e-8)  # Assume p \in (0, 1)
    denom = lambdas*p_current[:,None] + (1 - lambdas)*m_shifted
    dl_dp = np.sum(lambdas*((r/denom) - (1 - r)/(1 - denom)), axis=-1)
    return dl_dp

x0 = np.ones(N)*0.5  # Initial guess
ml = np.zeros((N,n))  # Initialize output
bounds = (1e-8, 1 - 1e-8)  # Assume p \in (0, 1)

# Break down along N if memory an issue
num_splits = 20
split_size = int(np.ceil(N/num_splits))
maxiter = 200

for j in range(num_splits):
    split_low, split_high = j*split_size, (j+1)*split_size
    if j + 1 == num_splits:
        split_high = N
    
    # Only calculate MLE for num_interactions that get plotted.
    for i in tqdm(plot_idx):
        pred = np.clip(
            newton(
                f, 
                x0[split_low:split_high], 
                args=(R[split_low:split_high,:i+1], M_prev[split_low:split_high,:i+1], lambdas[:i+1]), 
                maxiter=maxiter
            ),
            *bounds)
        ml[split_low:split_high,i] = pred

In [ ]:
# OPTIONAL: Comment out models if any should be excluded due to memory limitations
include_all = {
    "method_1": M
}
include_specific = {
    "method_2": p_hat,
    "method_3": p_hat_2,
    "method_4": ml
}

In [ ]:
data = {
    **{k: v.mean(axis=0) for k, v in include_all.items()},
    **{k: v.mean(axis=0) for k, v in include_specific.items()}
}

ci = {
    **{k:(np.percentile(v, 5, axis=0), np.percentile(v,100-5,axis=0)) for k, v in include_all.items()},
    **{k:(np.percentile(v, 5, axis=0), np.percentile(v,100-5,axis=0)) for k, v in include_specific.items()}
}

In [ ]:
# OPTIONAL: Run this cell if saving or loading current predictions
data_root = "/your/path/to/store(d)/data"
data_path = os.path.join(data_root, graph_name)
os.makedirs(data_path, exist_ok=True)

In [ ]:
# OPTIONAL: Run this cell to save current predictions to disk
for k, a in data.items():
    array_path = os.path.join(data_path, f"{k}_mean.npy")
    np.save(array_path, a)

for k,a in ci.items():
    array_path = os.path.join(data_path, f"{k}_ci.npy")
    np.save(array_path, np.stack(ci[k]))

In [ ]:
# OPTIONAL: Run this cell to load data/predictions from disk
include_all = ["method_1"]
include_specific = ["method_2", "method_3", "method_4"]

data = {}
ci = {}

for k in include_all + include_specific:
    mean_path = os.path.join(data_path, f"{k}_mean.npy")
    ci_path = os.path.join(data_path, f"{k}_ci.npy")
    data[k] = np.load(mean_path)
    ci[k] = tuple(x[0] for x in np.split(np.load(ci_path), 2))

In [ ]:
legend_info = {}
for k in data:
    legend_info[k] = {
        'linestyle': styles[k],
        'color': colours[k],
        'markersize': 12,
        'fillstyle': 'none',
        'label': name_mapping[k],
        'linewidth': linewidths[k]
    }

In [ ]:
# Iterate over models (except sample mean, which is included in every figure).
# Create a separate figure for every model
for k in include_specific:
    y = data[k]
    fig = plt.figure(figsize=(7.38/1.5, 1.25*2), linewidth=0.5)
    fig.tight_layout()
    plt.ioff()
    plt.xscale('log')
    plt.gca().yaxis.set_ticks_position('both')
    plt.gca().xaxis.set_ticks_position('both')
    plt.xlim(1, n)
    plt.xticks(10**np.arange(np.log10(n)+1))
    plt.ylim(0, 1.)
    
    # Plot the confidence interval for the current model
    y_min, y_max = ci[k]
    plt.fill_between(
        x_values[plot_idx], y_min[plot_idx], y_max[plot_idx],
        alpha=alpha if k != "method_4" else alpha-0.1,
        color=legend_info[k]["color"],
        zorder=3
    )

    # Bolden the confidence interval lines
    plt.plot(x_values[plot_idx], y_min[plot_idx], color=legend_info[k]["color"], alpha=0.3)
    plt.plot(x_values[plot_idx], y_max[plot_idx], color=legend_info[k]["color"], alpha=0.3)

    # Plot the mean for the current model
    plt.plot(x_values[plot_idx], y[plot_idx], zorder=2, **legend_info[k])
    
    # Plot the mean and the confidence interval of the sample mean in the background
    for _k in include_all:
        _y = data[_k]
        _y_min, _y_max = ci[_k]
        plt.fill_between(
            x_values[plot_idx], _y_min[plot_idx], _y_max[plot_idx],
            alpha=alpha,
            color=legend_info[_k]["color"],
            zorder=2
        )
        plt.plot(x_values[plot_idx], _y[plot_idx], zorder=1, **legend_info[_k])
    
    plt.savefig('./%s_%s.pdf' % (graph_name, name_mapping[k].replace(' ', '_')), bbox_inches='tight', pad_inches=0)     

In [ ]:
# Plot the legend as a separate figure
figlegend = plt.figure(figsize=(0.5,0.5))
ncol = len(data)
figlegend.legend(handles=[mlines.Line2D([], [], **l) for l in legend_info.values()],
               fontsize=18,
               loc='center',
               ncol=ncol,
               frameon=False,
               borderaxespad=0,
               borderpad=0,
               labelspacing=0.2,
               columnspacing=1.)
figlegend.savefig(f'./{graph_name}_legend.pdf', bbox_inches='tight')